# ETL Project - Scraping of desired Data Scientist skills requested in Job            Listings across United States

In [547]:
import sys

!{sys.executable} -m pip install html.parser
import html.parser
from html.parser import HTMLParser

import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

import unicodedata
import string

!{sys.executable} -m pip install pattern3
import pattern3

import numpy as np
import pandas as pd

import psycopg2 as ps
import psycopg2.extras
import os

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pprint import pprint

# Plotting tools
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [548]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Read In Data Scientist Position data from CSV file 

In [549]:
#csv_file_all_data = "../Resources/tstdata.csv"
csv_file_all_data = "../Resources/alldata.csv"
DS_jobs_all_data_df = pd.read_csv(csv_file_all_data,index_col=None, na_values=['NA'],sep=',')
DS_jobs_all_data_df.head(10)

,position,company,description,reviews,location
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA"
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA"
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303"
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA"
5,Manager of Data Engineering,McKinsey & Company,Qualifications\nBachelor’s degree in Computer ...,385.0,"Atlanta, GA 30318"
6,"Product Specialist - Periscope, New Ventures",McKinsey & Company,Qualifications\nBachelor’s degree\n5-7 years o...,385.0,"Atlanta, GA 30318"
7,"Junior to Mid-level Engineer, Geologist or Env...",Wood,Overview / Responsibilities\nWood Environment ...,899.0,"Atlanta, GA"
8,Analyst - CIB Credit Research,SunTrust,Works closely with senior CIB professionals. P...,3343.0,"Atlanta, GA"
9,Senior Associate - Cognitive Data Scientist Na...,KPMG,Known for being a great place to work and buil...,4494.0,"Atlanta, GA 30338"


# Remove the "reviews" column from the dataframe

In [550]:
#Create new data with selected columns- position, company, description, location
DS_jobs_df = DS_jobs_all_data_df[["position", "company","description","location"]].copy()
DS_jobs_df.head(6)

,position,company,description,location
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,"Atlanta, GA 30301"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...","Atlanta, GA"
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...","Atlanta, GA"
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,"Atlanta, GA 30303"
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,"Atlanta, GA"
5,Manager of Data Engineering,McKinsey & Company,Qualifications\nBachelor’s degree in Computer ...,"Atlanta, GA 30318"


# Transform location into City, State,  Zip

In [551]:
#Transformation Step: Split location into City, "State Zip"
#CityStateZip_df = new_DS_jobs_all_data_df["location"].str.extract('(?P<City>[A-Z ]{1,2}),(?P<State>[A-Z]*$) (?P<Zipcode>\d{5}){1,1}', expand=True)                                     
#CityStateZip_df = new_DS_jobs_all_data_df["location"].str.extract('(?P<City>([A-Z ]\S+){2,})', expand=True)
DS_jobs_df['city'], DS_jobs_df['state']  = DS_jobs_df["location"].str.split(', ',1).str
DS_jobs_df['zip'] = DS_jobs_df['state'].str[2:]
DS_jobs_df['state'] = DS_jobs_df['state'].str[:2]
DS_jobs_df.head()

,position,company,description,location,city,state,zip
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,"Atlanta, GA 30301",Atlanta,GA,30301
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...","Atlanta, GA",Atlanta,GA,
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...","Atlanta, GA",Atlanta,GA,
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,"Atlanta, GA 30303",Atlanta,GA,30303
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,"Atlanta, GA",Atlanta,GA,


# Drop column location from the dataframe

In [552]:
DS_jobs_df.drop(columns=["location"], inplace=True)
DS_jobs_df.head()

,position,company,description,city,state,zip
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,Atlanta,GA,30301
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",Atlanta,GA,
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",Atlanta,GA,
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,Atlanta,GA,30303
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,Atlanta,GA,


# Connect to the DataScientist_DB for data loading

In [553]:
#Connect to the DataScientist_DB
try: 
    conn = ps.connect(user = "postgres", 
                                  password = "firstCls1", 
                                  host = "localhost", 
                                  port = "5432", 
                                  database = "DataScientist_DB") 
 
    cursor = conn.cursor()
    # Print PostgreSQL version 
    cursor.execute("SELECT version();") 
    record = cursor.fetchone() 
    print("You are connected to - ", record,"\n") 

except (Exception, ps.Error) as error : 
    print ("Error while connecting to PostgreSQL", error) 


You are connected to -  ('PostgreSQL 11.4, compiled by Visual C++ build 1914, 64-bit',) 



# Load DS table with position details from the new_DS_jobs_all_data_df

In [554]:
#for each position in the dataframe load it into the ds_tbl table
df_columns = list(DS_jobs_df)
# create (col1,col2,...)
columns = ",".join(df_columns)
columns

'position,company,description,city,state,zip'

In [555]:
# create VALUES('%s', '%s",...) one '%s' per column
values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
values

'VALUES(%s,%s,%s,%s,%s,%s)'

In [556]:
#create INSERT INTO table (columns) VALUES('%s',...)
table = "ds_tbl"
insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
insert_stmt

'INSERT INTO ds_tbl (position,company,description,city,state,zip) VALUES(%s,%s,%s,%s,%s,%s)'

# Perform Batch Load of Data Scientist Positions

In [557]:
cursor = conn.cursor()
ps.extras.execute_batch(cursor, insert_stmt, DS_jobs_df.values)
conn.commit()

# Read In GEO Demographics data from CSV file 

In [558]:
geo_csv_file_data = "../Resources/14zpallagi.csv"
geo_df = pd.read_csv(geo_csv_file_data)
geo_df.head()


,STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,850050,481840,115070,240450,479900,1401930,...,389850,324575,0,0,0,0,62690,47433,744910,1964826
1,1,AL,0,2,491370,200750,150290,125560,281350,1016010,...,397110,950446,0,0,0,0,70780,101969,413790,1177400
2,1,AL,0,3,259540,75820,142970,34070,156720,589190,...,250230,1319641,0,0,0,0,62170,132373,192050,538160
3,1,AL,0,4,164840,26730,125410,10390,99750,423300,...,163580,1394913,0,0,0,0,45120,124048,115470,375882
4,1,AL,0,5,203650,18990,177070,5860,122670,565930,...,203050,3655700,610,135,270,66,81180,387298,114380,448442


# Load GEO Data table with GEO details from the 2015 Census

In [559]:
#for each position in the dataframe load it into the ds_tbl table
geo_df_columns = list(geo_df)
# create (col1,col2,...)
geocolumns = ",".join(geo_df_columns)
geocolumns

'STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N05780,A05780,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N85770,A85770,N85775,A85775,N09750,A09750,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N11560,A11560,N06500,A06500,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902'

In [560]:
# create VALUES('%s', '%s",...) one '%s' per column
#geovalues = "VALUES({})".format(",".join(["%s" for _ in geocolumns])) 
geovalues = 'VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'
geovalues

'VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'

In [561]:
#create INSERT INTO table (columns) VALUES('%s',...)
table = "geo_population_tbl"
insert_stmt = "INSERT INTO {} ({}) {}".format(table,geocolumns,geovalues)
insert_stmt

'INSERT INTO geo_population_tbl (STATEFIPS,STATE,zipcode,agi_stub,N1,MARS1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,A00100,N02650,A02650,N00200,A00200,N00300,A00300,N00600,A00600,N00650,A00650,N00700,A00700,N00900,A00900,N01000,A01000,N01400,A01400,N01700,A01700,SCHF,N02300,A02300,N02500,A02500,N26270,A26270,N02900,A02900,N03220,A03220,N03300,A03300,N03270,A03270,N03150,A03150,N03210,A03210,N03230,A03230,N03240,A03240,N04470,A04470,A00101,N18425,A18425,N18450,A18450,N18500,A18500,N18300,A18300,N19300,A19300,N19700,A19700,N04800,A04800,N05800,A05800,N09600,A09600,N05780,A05780,N07100,A07100,N07300,A07300,N07180,A07180,N07230,A07230,N07240,A07240,N07220,A07220,N07260,A07260,N09400,A09400,N85770,A85770,N85775,A85775,N09750,A09750,N10600,A10600,N59660,A59660,N59720,A59720,N11070,A11070,N10960,A10960,N11560,A11560,N06500,A06500,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,

In [562]:
geo_df.values

array([[1, 'AL', 0, ..., 47433, 744910, 1964826],
       [1, 'AL', 0, ..., 101969, 413790, 1177400],
       [1, 'AL', 0, ..., 132373, 192050, 538160],
       ...,
       [56, 'WY', 99999, ..., 2052, 1780, 5872],
       [56, 'WY', 99999, ..., 7835, 1630, 7139],
       [56, 'WY', 99999, ..., 33184, 260, 38860]], dtype=object)

# Perform Batch Load of GEO Population Data into Database

In [563]:
ps.extras.execute_batch(cursor, insert_stmt, geo_df.values)
conn.commit()

# Begin Extracting, Transforming and Loading Desired Data Scientist Skills

# SQL - Retrieve position ids

In [564]:
pos_id_df = pd.read_sql_query('select pos_id from ds_tbl',conn)
pos_id_df.head()

,pos_id
0,202041
1,202042
2,202043
3,202044
4,202045


Remove EOL/CRLF characters from descriptions resulting in one string for scraping.

In [565]:
desc_df = pd.read_sql_query("select regexp_replace(description, E'[\\n\\r]+', ' ', 'g' ) as description from ds_tbl",conn)
desc_df.head()

,description
0,The Opportunity Denali Therapeutics is dedicat...
1,SUMMARY: MyoKardia is seeking a talented and h...
2,Join us as we pursue our disruptive new vision...
3,The Role We are seeking machine learning devel...
4,"As a Senior Data Analyst, you will work with P..."


# Concatenate position ids list as the first column in the DS_jobs_df

In [566]:
id_desc_df = pd.concat((pos_id_df, desc_df), axis='columns')
id_desc_df.head()

,pos_id,description
0,202041,The Opportunity Denali Therapeutics is dedicat...
1,202042,SUMMARY: MyoKardia is seeking a talented and h...
2,202043,Join us as we pursue our disruptive new vision...
3,202044,The Role We are seeking machine learning devel...
4,202045,"As a Senior Data Analyst, you will work with P..."


Set discription field values to lowercase

In [567]:
id_desc_df['description'] = id_desc_df['description'].str.lower() 
id_desc_df.head(10)

,pos_id,description
0,202041,the opportunity denali therapeutics is dedicat...
1,202042,summary: myokardia is seeking a talented and h...
2,202043,join us as we pursue our disruptive new vision...
3,202044,the role we are seeking machine learning devel...
4,202045,"as a senior data analyst, you will work with p..."
5,202265,wish is looking for an exceptional pricing dat...
6,202046,about irhythm: irhythm is a leading digital he...
7,202047,wish is looking for an experienced senior quan...
8,202048,more than 7 billion people depend on farms for...
9,202049,summary: data engineer is responsible for silu...


# Begin Natural Language Processing to Scrape Data Scienctist Skills

Begin by defining Data Scienctist Skills (for future development, implement Machine Learning to identify skills learning from all the previous skills identified)

In [568]:
skillsfile = "../Resources/skills_list.csv"
skills_match_list_df = pd.read_csv(skillsfile)
skills_match_list_df.head(25)

,skills
0,actionable recommendations
1,adapt quickly
2,alteryx
3,analytical
4,analytical tools
5,analyze data
6,audits
7,azure
8,b2b applications
9,bachelor


Initiate Natural Language Processing to Scrape Data Scienctist Skills

In [569]:
wnl = WordNetLemmatizer()
html_parser = HTMLParser()

Defining the dictionary of word contractions:

In [570]:
contraction_mapping = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

Select the list of stopwords from NLTK and amend it by adding more stopwords to it:

In [571]:
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also','would']

Split text into word tokens:

In [572]:
def tokenize_text(position_desc):
    tokens = nltk.word_tokenize(position_desc) 
    tokens = [token.strip() for token in tokens]
    return tokens

    text = tokenize_text(position_desc)

Expand contractions:

In [573]:
def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
 
    text = expand_contractions(text, contraction_mapping)

Annotate text tokens with Part-Of-Speach tags:

In [574]:
def pos_tag_text(text_tokens):
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None  
    tagged_text = nltk.pos_tag(text_tokens)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]
    return tagged_lower_text

    text = pos_tag_text(tokenize_text(text))

Lemmatize text based on Part-Of-Speech (POS) tags:

In [575]:
def lemmatize_text(text):
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word                     
                         for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text
 
    text = lemmatize_text(tokenize_text(text))

Remove special characters, such as punctuation marks:

In [576]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text 

    text = remove_special_characters(text)

Get rid of stopwords:

In [577]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

    text = remove_stopwords(text)

Remove all non-text characters (numbers, etc.):

In [578]:
def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

    text = keep_text_characters(test_text)

Clean up HTML markups: 

In [579]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)
    
def strip_html(text):
    html_stripper = MLStripper()
    html_stripper.feed(text)
    return html_stripper.get_data()

    text = strip_html(text)

Removing accents from characters:

In [580]:
def normalize_accented_characters(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8')
    return text

    text = normalize_accented_characters(text)

Using the lemma tokenized words try to match each word to the skills_match_list df to identify desired Data Scientist skills to load to the skills table. 

In [581]:
def identify_skills(text):
    skills_list = []
    
    #Compare each word in the normalized position description to skills list: if matched append skill to skills_list
    for word_tokens in text:
        isSkill = word_tokens in skills_match_list_df.values
        if isSkill:
            skills_list.append(word_tokens)
            isSkill = False
    
    #Remove duplicate skills from list
    normalize_skills_list = list(set(skills_list))
    
    #return normalized_skills_list
    return normalize_skills_list
    

Main process loop through all position descriptions to scrape all desired skills

In [582]:
def normalize_skills_list(corpus, only_text_chars=True):

    normalized_skills_list = []
    index = 0
    #for text in corpus_desc:
    index += 1 
    text = normalize_accented_characters(corpus)
    text = html.unescape(text)
    text = strip_html(text)
    text = expand_contractions(text, contraction_mapping)
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    text = tokenize_text(text)
    
    normalized_skills_list = identify_skills(text)    
    return normalized_skills_list

In [583]:
def load_skills(hld_pos_id, skills_list):
    skills_list_df = pd.DataFrame(skills_list, columns = ['skill'])
    skills_list_df['pos_id']=hld_pos_id
        
    print("skill list df: ", skills_list_df)
    
    #Generate the sql insert statement to load all the skills identifed for the current DS position
    insert_stmt = "INSERT INTO skills_tbl (skill, pos_id) VALUES(%s, %s);"
    
    #Load skills into the skills database table
    ps.extras.execute_batch(cursor, insert_stmt, skills_list_df.values)
    conn.commit()    

In [584]:
for index, row in id_desc_df.iterrows():
    hld_pos_id = row['pos_id']
    corpus = []
    corpus = row['description']
    skills_list = normalize_skills_list(corpus)
    load_skills(hld_pos_id, skills_list)
    


skill list df:           skill  pos_id
0  development  202041
1    excellent  202041
2       design  202041
3   principles  202041
4     optimize  202041
5     datasets  202041
6   developing  202041
7   analytical  202041
skill list df:            skill  pos_id
0   development  202042
1    technology  202042
2   statistical  202042
3             r  202042
4      database  202042
5        python  202042
6     excellent  202042
7        design  202042
8   information  202042
9      datasets  202042
10    integrity  202042
11   developing  202042
12   hypotheses  202042
13        focus  202042
14   innovative  202042
15      methods  202042
skill list df:            skill  pos_id
0        client  202043
1   development  202043
2   maintaining  202043
3   responsible  202043
4       deliver  202043
5        trends  202043
6    management  202043
7      insights  202043
8        design  202043
9         azure  202043
10  information  202043
11     datasets  202043
12      experts  202043
1

skill list df:           skill  pos_id
0  development  202065
1  maintaining  202065
2  responsible  202065
3       python  202065
4   management  202065
5    excellent  202065
6  performance  202065
7     optimize  202065
8     creating  202065
9        focus  202065
skill list df:             skill  pos_id
0         import  202066
1       database  202066
2    development  202066
3      marketing  202066
4        process  202066
5         trends  202066
6   organization  202066
7      excellent  202066
8     proficient  202066
9         issues  202066
10   information  202066
skill list df:            skill  pos_id
0   development  202067
1    technology  202067
2        python  202067
3     excellent  202067
4      insights  202067
5        design  202067
6        desire  202067
7      optimize  202067
8      datasets  202067
9       experts  202067
10       deploy  202067
skill list df:            skill  pos_id
0   development  202266
1      database  202266
2             r  202266

skill list df:           skill  pos_id
0     database  195567
1  development  195567
2   technology  195567
3  statistical  195567
4   management  195567
5       design  195567
6  information  195567
skill list df:                skill  pos_id
0            hadoop  195568
1   entrepreneurial  195568
2        innovative  195568
3       development  195568
4         marketing  195568
5       performance  195568
6        developing  195568
7          database  195568
8           deliver  195568
9      organization  195568
10            sales  195568
11     optimization  195568
12           desire  195568
13           design  195568
14            azure  195568
15          experts  195568
16       analytical  195568
17           client  195568
18              sql  195568
19          alteryx  195568
20         insights  195568
21      information  195568
skill list df:                skill  pos_id
0        technology  195569
1   entrepreneurial  195569
2        management  195569
3         pr

skill list df:             skill  pos_id
0         hadoop  195592
1     technology  195592
2         python  195592
3     management  195592
4      reporting  195592
5      integrity  195592
6        testing  195592
7    development  195592
8    responsible  195592
9    performance  195592
10      database  195592
11      modeling  195592
12           sas  195592
13  optimization  195592
14     excellent  195592
15        design  195592
16         azure  195592
17    analytical  195592
18      findings  195592
19        client  195592
20   statistical  195592
21             r  195592
22           sql  195592
23     databases  195592
24   information  195592
25       methods  195592
skill list df:            skill  pos_id
0         focus  195593
1      database  195593
2   development  195593
3             r  195593
4       process  195593
5       deliver  195593
6        python  195593
7        design  195593
8     integrity  195593
9      creating  195593
10   delivering  195593
11   

skill list df:             skill  pos_id
0         hadoop  195614
1     technology  195614
2         python  195614
3     management  195614
4     hypotheses  195614
5     innovative  195614
6    development  195614
7      marketing  195614
8         trends  195614
9       modeling  195614
10       process  195614
11       deliver  195614
12           sas  195614
13  optimization  195614
14     excellent  195614
15   statistical  195614
16             r  195614
17   operational  195614
18      insights  195614
19   information  195614
20       tableau  195614
21       methods  195614
skill list df:            skill  pos_id
0    management  195615
1     reporting  195615
2    innovative  195615
3   responsible  195615
4     marketing  195615
5        trends  195615
6   performance  195615
7      modeling  195615
8       process  195615
9       deliver  195615
10          sas  195615
11    excellent  195615
12   analytical  195615
13       client  195615
14  statistical  195615
15       

...

skill list df:            skill  pos_id
0    technology  202373
1        hadoop  202373
2       process  202373
3  organization  202373
4        python  202373
5  optimization  202373
6   performance  202373
7      datasets  202373
8    analytical  202373
skill list df:           skill  pos_id
0  development  202374
1   technology  202374
2    marketing  202374
3     insights  202374
4       design  202374
skill list df:           skill  pos_id
0      deliver  202375
1       python  202375
2   management  202375
3  operational  202375
4    excellent  202375
5       design  202375
6  performance  202375
7   principles  202375
8      testing  202375
skill list df:            skill  pos_id
0   development  202376
1       methods  202376
2   responsible  202376
3    technology  202376
4       deliver  202376
5        hadoop  202376
6        python  202376
7      patterns  202376
8    management  202376
9     excellent  202376
10       design  202376
11  performance  202376
12   deployment 

skill list df:            skill  pos_id
0   development  202403
1    technology  202403
2             r  202403
3       process  202403
4    management  202403
5       testing  202403
6     excellent  202403
7        design  202403
8   performance  202403
9    delivering  202403
10      methods  202403
skill list df:             skill  pos_id
0     technology  202408
1      marketing  202408
2        process  202408
3        deliver  202408
4     management  202408
5   optimization  202408
6      reporting  202408
7       insights  202408
8    performance  202408
9     delivering  202408
10    developing  202408
11        deploy  202408
12    analytical  202408
13    innovative  202408
skill list df:             skill  pos_id
0    development  202409
1       modeling  202409
2        process  202409
3         python  202409
4   optimization  202409
5         issues  202409
6         design  202409
7       optimize  202409
8       creating  202409
9     innovative  202409
10       testi

skill list df:            skill  pos_id
0        client  202433
1   development  202433
2    technology  202433
3      database  202433
4       process  202433
5      patterns  202433
6    proficient  202433
7        design  202433
8   performance  202433
9   information  202433
10   developing  202433
11     findings  202433
12      testing  202433
13     bachelor  202433
skill list df:         skill  pos_id
0   patterns  202434
1    deliver  202434
2  excellent  202434
3     design  202434
skill list df:            skill  pos_id
0   development  202435
1    technology  202435
2       deliver  202435
3  organization  202435
4      patterns  202435
5  optimization  202435
6     excellent  202435
7        design  202435
8   performance  202435
9       experts  202435
skill list df:           skill  pos_id
0  development  202479
1  responsible  202479
2    marketing  202479
3      process  202479
4     patterns  202479
5    excellent  202479
6   analytical  202479
7      testing  202479


skill list df:            skill  pos_id
0   development  202482
1      modeling  202482
2   statistical  202482
3             r  202482
4  optimization  202482
5       experts  202482
6      creating  202482
7    innovative  202482
8       methods  202482
skill list df:           skill  pos_id
0  development  202483
1      deliver  202483
2    excellent  202483
3       design  202483
4   developing  202483
skill list df:             skill  pos_id
0    development  202484
1         hadoop  202484
2    statistical  202484
3              r  202484
4            sql  202484
5          sales  202484
6   organization  202484
7            sas  202484
8     management  202484
9       insights  202484
10    deployment  202484
11      creating  202484
12    delivering  202484
13         focus  202484
14    analytical  202484
15      findings  202484
skill list df:             skill  pos_id
0     technology  202485
1         hadoop  202485
2         python  202485
3     management  202485
4      r

skill list df:             skill  pos_id
0       database  202511
1         hadoop  202511
2     technology  202511
3        deliver  202511
4        process  202511
5   organization  202511
6         design  202511
7          azure  202511
8    information  202511
9       creating  202511
10    developing  202511
11    innovative  202511
skill list df:             skill  pos_id
0     technology  202512
1    responsible  202512
2              r  202512
3            sql  202512
4         python  202512
5   optimization  202512
6      excellent  202512
7         issues  202512
8     proficient  202512
9    performance  202512
10       experts  202512
11        deploy  202512
skill list df:           skill  pos_id
0  development  202513
1   technology  202513
2     modeling  202513
3       python  202513
4       design  202513
skill list df:            skill  pos_id
0    technology  202514
1      modeling  202514
2   statistical  202514
3        hadoop  202514
4         sales  202514
5  o

In [186]:
connection.close()